# Load data

In [1]:
import pandas as pd
import numpy as np
from utils import * 

In [2]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

In [3]:
data = pd.read_csv('../input/new_data.csv')
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [4]:
data = transformDf(data)
train = transformDf(train)
test = transformDf(test)

In [5]:
data['ActivityID'].unique()

array(['1.0', '8.0', '6.0', '3.0', '9.0', '2.0', '4.0', '5.0', '7.0'], dtype=object)

In [6]:
data['TimeSinceMidnight'].unique()

array([ 60938.,  60953.,  62152., ...,  77531.,   1811.,   7660.])

In [7]:
data.shape, train.shape, test.shape #13710, 9181, 4529

((13710, 7), (9181, 7), (4529, 7))

# Parameters

Parameters for preprocessing

In [8]:
groupByCase = data.groupby(['CaseID'])

In [9]:
# define the denominator for normalization
divisor = data['Duration'].mean()
divisor2 = data['CumDuration'].mean()

In [10]:
#find len of longest case
maxlen = findLongestLength(groupByCase)
maxlen

15

In [11]:
#define number of features
num_features = len(data['ActivityID'].unique()) + 5
num_features

14

In [12]:
unique_chars = data['ActivityID'].unique().tolist()
target_chars = unique_chars + ['EOS']

In [13]:
chartoindice = char_indice_dict(unique_chars)
targetchartoindice = char_indice_dict(target_chars)

In [14]:
targetchartoindice

{'1.0': 0,
 '2.0': 5,
 '3.0': 3,
 '4.0': 6,
 '5.0': 7,
 '6.0': 2,
 '7.0': 8,
 '8.0': 1,
 '9.0': 4,
 'EOS': 9}

# Train

## Input

In [15]:
train_groupByCase = train.groupby(['CaseID'])

In [16]:
sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4 = getFeature(train_groupByCase)

In [17]:
len(sentences), len(sentences_t), len(sentences_t2), len(sentences_t3), len(sentences_t4)

(9181, 9181, 9181, 9181, 9181)

In [18]:
X = vectorizeInput(train_groupByCase, maxlen, num_features, chartoindice, divisor, divisor2)

In [19]:
X.shape

(9181, 15, 14)

## Output

In [20]:
next_chars, next_chars_t, next_chars_t2, next_chars_t3, next_chars_t4 = getOutput(train_groupByCase)

In [21]:
len(next_chars), len(next_chars_t), len(next_chars_t2), len(next_chars_t3), len(next_chars_t4)

(9181, 9181, 9181, 9181, 9181)

### y_a

In [22]:
next_chars[:10]

['8.0', '6.0', 'EOS', '8.0', '6.0', 'EOS', '8.0', '6.0', 'EOS', '8.0']

In [23]:
y_a = one_hot_encode(train_groupByCase, targetchartoindice)

In [24]:
y_a.shape

(9181, 10)

In [25]:
y_a[:10]

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

### y_t 

In [26]:
next_chars_t = np.asarray(next_chars_t)

In [27]:
y_t = normalize(train_groupByCase, divisor)

In [28]:
y_t

array([  7.11185218e-05,   8.24970111e-01,   0.00000000e+00, ...,
         7.11185218e-05,   4.12487426e-04,   0.00000000e+00])

In [29]:
y_t.shape

(9181,)

# Test

In [30]:
test_groupByCase = test.groupby(['CaseID'])

In [31]:
df_test = pd.DataFrame(columns=['CaseID', 'ActivityID', 'CompleteTimestamp', 'Duration', 'CumDuration', 'TimeSinceMidnight', 'WeekDay'])
for case, group in test_groupByCase:
    if group.shape[0] > 1:     
        df_test = df_test.append(group)

In [32]:
df_test.shape #No case with one activity

(4529, 7)

## Input

In [33]:
sentences, sentences_t, sentences_t2, sentences_t3, sentences_t4 = getFeature(test_groupByCase)

In [34]:
len(sentences), len(sentences_t), len(sentences_t2), len(sentences_t3), len(sentences_t4)

(4529, 4529, 4529, 4529, 4529)

In [35]:
X_test = vectorizeInput(test_groupByCase, maxlen, num_features, chartoindice, divisor, divisor2)

In [36]:
X.shape

(9181, 15, 14)

## Output

In [37]:
next_chars, next_chars_t, next_chars_t2, next_chars_t3, next_chars_t4 = getOutput(test_groupByCase)

In [38]:
len(next_chars), len(next_chars_t), len(next_chars_t2), len(next_chars_t3), len(next_chars_t4)

(4529, 4529, 4529, 4529, 4529)

### y_a

One-hot encode for y_a

In [39]:
next_chars[:10]

['8.0', '6.0', 'EOS', '1.0', '8.0', '9.0', '8.0', '6.0', 'EOS', '8.0']

In [40]:
y_a_test = one_hot_encode(test_groupByCase, targetchartoindice)

In [41]:
y_a_test.shape

(4529, 10)

In [42]:
y_a[:10]

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.],
       [ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]], dtype=float32)

### y_t

Scale/Normalize data. This can be done by using [sklearn](http://scikit-learn.org/stable/modules/preprocessing.html#preprocessing-scaler)

In [43]:
y_t_test = normalize(test_groupByCase, divisor)

In [44]:
y_t_test

array([ 1.3322443 ,  1.21594656,  0.        , ...,  2.96890433,
        0.02773148,  0.        ])

In [45]:
y_t_test.shape

(4529,)

# Save data

In [46]:
import pickle

In [47]:
with open('../input/parameters.pkl', 'wb') as f:
    pickle.dump(maxlen, f, protocol=2)
    pickle.dump(num_features, f, protocol=2)
    pickle.dump(chartoindice, f, protocol=2)
    pickle.dump(targetchartoindice, f, protocol=2)
    pickle.dump(divisor, f, protocol=2)
    pickle.dump(divisor2, f, protocol=2)

In [48]:
with open('../input/preprocessed_data.pkl', 'wb') as f:
    pickle.dump(X, f, protocol=2)
    pickle.dump(y_a, f, protocol=2)
    pickle.dump(y_t, f, protocol=2)
    pickle.dump(X_test, f, protocol=2)
    pickle.dump(y_a_test, f, protocol=2)
    pickle.dump(y_t_test, f, protocol=2)

# Test model

In [49]:
from keras.models import Sequential, Model
from keras.layers.core import Dense
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.layers import Input
from keras.utils.data_utils import get_file
from keras.regularizers import WeightRegularizer
from keras.optimizers import Nadam
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers.normalization import BatchNormalization
from theano.ifelse import ifelse

Using Theano backend.


In [50]:
# build the model: 
print('Build model...')
main_input = Input(shape=(maxlen, num_features), name='main_input')
# train a 2-layer LSTM with one shared layer
l1 = LSTM(100, consume_less='gpu', init='glorot_uniform', return_sequences=True, dropout_W=0.2)(main_input) # the shared layer
b1 = BatchNormalization()(l1)
l2_1 = LSTM(100, consume_less='gpu', init='glorot_uniform', return_sequences=False, dropout_W=0.2)(b1) # the layer specialized in activity prediction
b2_1 = BatchNormalization()(l2_1)
l2_2 = LSTM(100, consume_less='gpu', init='glorot_uniform', return_sequences=False, dropout_W=0.2)(b1) # the layer specialized in time prediction
b2_2 = BatchNormalization()(l2_2)
act_output = Dense(len(target_chars), activation='softmax', init='glorot_uniform', name='act_output')(b2_1)
time_output = Dense(1, init='glorot_uniform', name='time_output')(b2_2)

model = Model(input=[main_input], output=[act_output, time_output])

opt = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=1e-08, schedule_decay=0.004, clipvalue=3)

model.compile(loss={'act_output':'categorical_crossentropy', 'time_output':'mae'}, optimizer=opt)
early_stopping = EarlyStopping(monitor='val_loss', patience=42)
model_checkpoint = ModelCheckpoint('output_files/models/model_{epoch:02d}-{val_loss:.2f}.h5', monitor='val_loss', verbose=0, save_best_only=True, save_weights_only=False, mode='auto')
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=0, mode='auto', epsilon=0.0001, cooldown=0, min_lr=0)

model.fit(X, {'act_output':y_a, 'time_output':y_t}, validation_split=0.2, verbose=2, callbacks=[early_stopping, model_checkpoint, lr_reducer], batch_size=maxlen, nb_epoch=500)

Build model...
Train on 7344 samples, validate on 1837 samples
Epoch 1/500
19s - loss: 2.1553 - act_output_loss: 0.9735 - time_output_loss: 1.1818 - val_loss: 1.7406 - val_act_output_loss: 0.6893 - val_time_output_loss: 1.0513
Epoch 2/500
18s - loss: 1.7593 - act_output_loss: 0.7159 - time_output_loss: 1.0434 - val_loss: 1.5959 - val_act_output_loss: 0.5990 - val_time_output_loss: 0.9969
Epoch 3/500
18s - loss: 1.7187 - act_output_loss: 0.6954 - time_output_loss: 1.0232 - val_loss: 1.5521 - val_act_output_loss: 0.5806 - val_time_output_loss: 0.9716
Epoch 4/500
18s - loss: 1.6923 - act_output_loss: 0.6757 - time_output_loss: 1.0166 - val_loss: 1.6068 - val_act_output_loss: 0.5818 - val_time_output_loss: 1.0249
Epoch 5/500
18s - loss: 1.6821 - act_output_loss: 0.6723 - time_output_loss: 1.0098 - val_loss: 1.5755 - val_act_output_loss: 0.5953 - val_time_output_loss: 0.9803
Epoch 6/500
18s - loss: 1.6637 - act_output_loss: 0.6590 - time_output_loss: 1.0047 - val_loss: 1.5628 - val_act_outp

18s - loss: 1.5820 - act_output_loss: 0.6014 - time_output_loss: 0.9806 - val_loss: 1.5134 - val_act_output_loss: 0.5632 - val_time_output_loss: 0.9502
Epoch 52/500
17s - loss: 1.5802 - act_output_loss: 0.6026 - time_output_loss: 0.9776 - val_loss: 1.5156 - val_act_output_loss: 0.5633 - val_time_output_loss: 0.9523
Epoch 53/500
18s - loss: 1.5793 - act_output_loss: 0.5980 - time_output_loss: 0.9812 - val_loss: 1.5165 - val_act_output_loss: 0.5648 - val_time_output_loss: 0.9517
Epoch 54/500
18s - loss: 1.5767 - act_output_loss: 0.5941 - time_output_loss: 0.9826 - val_loss: 1.5166 - val_act_output_loss: 0.5632 - val_time_output_loss: 0.9534
Epoch 55/500
18s - loss: 1.5807 - act_output_loss: 0.6007 - time_output_loss: 0.9801 - val_loss: 1.5141 - val_act_output_loss: 0.5638 - val_time_output_loss: 0.9503
Epoch 56/500
18s - loss: 1.5949 - act_output_loss: 0.6067 - time_output_loss: 0.9882 - val_loss: 1.5211 - val_act_output_loss: 0.5656 - val_time_output_loss: 0.9555
Epoch 57/500
18s - loss

In [51]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
main_input (InputLayer)          (None, 15, 14)        0                                            
____________________________________________________________________________________________________
lstm_1 (LSTM)                    (None, 15, 100)       46000       main_input[0][0]                 
____________________________________________________________________________________________________
batchnormalization_1 (BatchNorma (None, 15, 100)       400         lstm_1[0][0]                     
____________________________________________________________________________________________________
lstm_2 (LSTM)                    (None, 100)           80400       batchnormalization_1[0][0]       
___________________________________________________________________________________________